# Tutorial 4: Agents in LangChain

In this tutorial, we'll explore Agents in LangChain, which are autonomous entities capable of using tools and making decisions to accomplish tasks.

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import load_tools, initialize_agent, AgentType,create_react_agent, AgentExecutor
from langchain.tools import Tool,DuckDuckGoSearchRun
# from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
# from langchain_community.tools.playwright.utils import create_sync_playwright_browser
# from langgraph.prebuilt.chat_agent_executor import create_react_agent

# Load environment variables
load_dotenv()

# Initialize Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.7,
        model_kwargs={"top_p": 0.8, "seed": 1337}
    )

## 1. Understanding Agent Architecture

Agents in LangChain combine an LLM with a set of tools to perform tasks. The LLM decides which tools to use and how to interpret their outputs.

## 2. Exploring Different Types of Agents

### Zero-shot React Agent

In [ ]:
search = DuckDuckGoSearchRun()

def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = load_tools(["wikipedia", "llm-math"], llm=llm)

zero_shot_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

zero_shot_agent.invoke(
    "What is the square root of the year Pythagoras was born?"
)

/tmp/ipykernel_114402/2048912467.py:19: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  zero_shot_agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: To find the square root of the year Pythagoras was born, I first need to find the year Pythagoras was born. Then, I can use the calculator to find the square root of that year.

Action: wikipedia
Action Input: Pythagoras
Observation: Page: Pythagoras
Summary: Pythagoras of Samos (Ancient Greek: Πυθαγόρας; c. 570 – c. 495 BC) was an ancient Ionian Greek philosopher, polymath, and the eponymous founder of Pythagoreanism. His political and religious teachings were well known in Magna Graecia and influenced the philosophies of Plato, Aristotle, and, through them, the West in general. Knowledge of his life is clouded by legend; modern scholars disagree regarding Pythagoras's education and influences, but they do agree that, around 530 BC, he travelled to Croton in southern Italy, where he founded a school in which initiates were sworn to secrecy and lived a communal, ascetic lifestyle.
In antiquity, Pythagoras was credited with many mathemati

{'input': 'What is the square root of the year Pythagoras was born?',
 'output': 'The square root of the year Pythagoras was born is approximately 23.87.'}

In [18]:
#Create Web Search Tool
tools_web_search =[
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Useful for searching the internet for current information"
    )
]

### Conversational Agent

In [21]:
conversational_agent = initialize_agent(
    tools+tools_web_search,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

conversational_agent.invoke({"input":"Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.","chat_history":""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: theory of relativity
Observation: Page: Theory of relativity
Summary: The theory of relativity usually encompasses two interrelated physics theories by Albert Einstein: special relativity and general relativity, proposed and published in 1905 and 1915, respectively. Special relativity applies to all physical phenomena in the absence of gravity. General relativity explains the law of gravitation and its relation to the forces of nature. It applies to the cosmological and astrophysical realm, including astronomy.
The theory transformed theoretical physics and astronomy during the 20th century, superseding a 200-year-old theory of mechanics created primarily by Isaac Newton. It introduced concepts including 4-dimensional spacetime as a unified entity of space and time, relativity of simultaneity, kinematic and gravitational time dilation, and length contraction. In the field of ph

{'input': "Hello! I'd like to learn about the theory of relativity and then solve a math problem related to it.",
 'chat_history': '',
 'output': "Now that you have a basic understanding of the theory of relativity, let's solve a math problem related to it. What kind of math problem would you like to solve? Would you like to calculate time dilation, length contraction, or something else?"}

### STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION Agent

In [24]:
plan_and_execute_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

plan_and_execute_agent.run(
    "Find out who invented the telephone and calculate the number of years between that invention and the first moon landing."
)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to find out who invented the telephone and the year it was invented. Then, I can calculate the number of years between the invention of the telephone and the first moon landing.

Action:
```
{
  "action": "wikipedia",
  "action_input": "telephone invention"
}
```

Observation: Page: Invention of the telephone
Summary: The invention of the telephone was the culmination of work done by more than one individual, and led to an array of lawsuits relating to the patent claims of several individuals and numerous companies. Notable people included in this were Antonio Meucci, Philipp Reis, Elisha Gray and Alexander Graham Bell.

Page: History of the telephone
Summary: This history of the telephone chronicles the development of the electrical telephone, and includes a brief overview of its predecessors.  The first telephone patent was granted to Alexander Graham Bell in 1869.



Page: Reis telephone
Summary: The Re

'The invention of the telephone is credited to several individuals, including Alexander Graham Bell, who was granted the first telephone patent in 1869. The first moon landing occurred on July 20, 1969. Therefore, the number of years between the invention of the telephone and the first moon landing is 108 years.'

## 3. Creating Custom Tools for Agents

In [15]:
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

word_length_tool = Tool(
    name="WordLength",
    func=get_word_length,
    description="Useful for getting the length of a word"
)

custom_tools = [word_length_tool] + tools

custom_agent = initialize_agent(
    custom_tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

custom_agent.run(
    "What's the length of the word 'supercalifragilisticexpialidocious' and its square root?"
)



> Entering new AgentExecutor chain...
Thought: To find the length of the word 'supercalifragilisticexpialidocious', we can use the WordLength tool. To find the square root of the length, we can use the Calculator tool.

Action:
```
{
  "action": "WordLength",
  "action_input": "supercalifragilisticexpialidocious"
}
```

Observation: 34
Thought:To find the square root of the length of the word 'supercalifragilisticexpialidocious', we can use the Calculator tool with the previously calculated length as the input.

Action:
```
{
  "action": "Calculator",
  "action_input": "sqrt(34)"
}
```


Observation: Answer: 5.830951894845301
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The length of the word 'supercalifragilisticexpialidocious' is 34 and its square root is approximately 5.83."
}
```

> Finished chain.


"The length of the word 'supercalifragilisticexpialidocious' is 34 and its square root is approximately 5.83."

## 4. Implementing a Multi-tool Agent for Task Solving

In [25]:
# Combine all tools
all_tools = tools + tools_web_search + [word_length_tool]

multi_tool_agent = initialize_agent(
    all_tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

multi_tool_agent.run(
    """Please perform the following tasks:
    1. Find the current temperature in New York City.
    2. Calculate the square root of that temperature.
    3. Find a famous quote about weather and tell me how many characters it has.
    Provide a summary of your findings.
    """
)



> Entering new AgentExecutor chain...
Thought: To find the current temperature in New York City, I will use the DuckDuckGo Search tool.

Action:
```
{
  "action": "DuckDuckGo Search",
  "action_input": "current temperature in New York City"
}
```

Observation: Current conditions at New York City, Central Park (KNYC) Lat: 40.78°NLon: 73.97°WElev: 154ft. Fair. 52°F. 11°C. ... New York NY . Red Flag Warning November 12, 02:28pm until November 12, 06:00pm ... National Weather Service; New York, NY; 175 Brookhaven Avenue; Upton, NY 11973; Comments? Questions? Please Contact Us. NOAA National Weather Service. Current conditions at New York City, Central Park (KNYC) Lat: 40.78°NLon: 73.97°WElev: 154ft. Current conditions at New York City, Central Park (KNYC) Lat: 40.78° N Lon: 73.97° W Elev: 154 ft. Weather Underground provides local and long-range weather forecasts, weather reports, maps, and tropical weather conditions for the New York City area. Current weather in New York and forecast f

"The current temperature in New York City is 52°F. The square root of the temperature is 7.211102550927978. A famous quote about weather is 'There is no such thing as bad weather, only different kinds of good weather.' - John Ruskin. This quote has 76 characters."

## Conclusion

In this tutorial, we've explored Agents in LangChain, including different types of agents, creating custom tools, and implementing a multi-tool agent for complex task solving. Agents provide a powerful way to create autonomous systems that can leverage various tools and make decisions to accomplish tasks.